In [1]:
import torch
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
)

In [4]:
merged_model_dir = "merged-fine-tuned"

device_map = {"": 0}

In [5]:
merged_model = AutoModelForCausalLM.from_pretrained(
    merged_model_dir,
    low_cpu_mem_usage=True,
    return_dict=True,
    torch_dtype=torch.float16,
    device_map=device_map,
)

tokenizer = AutoTokenizer.from_pretrained(merged_model_dir)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [6]:
prompt = """What is your name?"""
input_ids = tokenizer(prompt, return_tensors="pt", truncation=True).input_ids.cuda()
# with torch.inference_mode():
outputs = merged_model.generate(input_ids=input_ids, max_new_tokens=100, do_sample=True, top_p=0.9,temperature=0.2,pad_token_id=tokenizer.eos_token_id)

print(f"Prompt:\n{prompt}")
print(f"Generated:\n{tokenizer.batch_decode(outputs)[0][len(prompt):]}")

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Prompt:
What is your name?
Generated:
ame? [/INST]Capstone.Capstone is an AI language model. What's your name? [/INST]Capstone.Capstone is an AI language model. What's your name?</s>
